In [2]:
# https://openrouteservice.org/
# https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/get

# username = dconly
# email = dconly@sacog.org

import requests
import os

import geopandas as gpd
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy
arcpy.env.overwriteOutput = True

api_key_source = r"C:\Users\dconly\GitRepos\GIS-tools\ORS\api2_DO_NOT_COMMIT.txt"
with open(api_key_source) as f:
    ors_api_key = f.readline()







In [2]:
file_gdb = r'I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb'
fc = 'BlockGroups2010'
gdf_bgs = gpd.GeoDataFrame.from_file(file_gdb, layer=fc, driver="OpenFileGDB")
gdf_bgs.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,Shape_Length,Shape_Area,geometry
0,06,017,031800,2,060170318002,Block Group 2,G5030,S,5363013.0,0.0,+38.6663472,-121.0456300,44120.610088,5.771996e+07,"MULTIPOLYGON (((6837199.297 2009428.472, 68371..."
1,06,017,030801,1,060170308011,Block Group 1,G5030,S,21172302.0,30483.0,+38.7183035,-120.9816213,73827.779033,2.281943e+08,"MULTIPOLYGON (((6850910.115 2018490.882, 68507..."
2,06,017,030801,2,060170308012,Block Group 2,G5030,S,40628317.0,128159.0,+38.7323424,-121.0257295,126155.325004,4.386383e+08,"MULTIPOLYGON (((6845078.938 2025006.203, 68450..."
3,06,017,030807,2,060170308072,Block Group 2,G5030,S,3541193.0,0.0,+38.6613873,-121.0170995,28049.260986,3.811249e+07,"MULTIPOLYGON (((6843688.528 2006343.118, 68437..."
4,06,017,030704,1,060170307041,Block Group 1,G5030,S,133130500.0,16859.0,+38.5829792,-120.9950293,211588.384842,1.433048e+09,"MULTIPOLYGON (((6826658.539 1986707.326, 68266..."


In [21]:
# Make isochrone around single point (NOT necessary to run for making line-based isochrone)
# https://openrouteservice.org/dev/#/api-docs/isochrones

travel_mode = "foot-walking"

orgn_lat = 38.59312635026946
orgn_lon = -121.4487934112549

max_time_mins = 15
max_time_sec = max_time_mins * 60


# IMPORTANT NOTE!!! CAN ENTER ARRAY OF MULTIPLE LAT/LONGS, SO COULD INSERT MULTIPLE POINTS TO GET LINE-BASED ISOCHRONE
body = {"locations":[[orgn_lon, orgn_lat]], "range":[max_time_sec], "range_type":"time"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': ors_api_key,
    'Content-Type': 'application/json; charset=utf-8'
}

call = requests.post(f'https://api.openrouteservice.org/v2/isochrones/{travel_mode}', json=body, headers=headers)

polygon_features = call.json()['features']
txt_test = call.text

In [28]:
import json
def json_to_sedf(in_json_str, k_features='features'):
    """Takes in a json string, loads it to dict, then converts to
    ESRI spatially-enabled dataframe (SEDF"""

    json_loaded = json.loads(in_json_str)

    if k_features in json_loaded.keys():
        gdf = gpd.GeoDataFrame.from_features(json_loaded[k_features])
        sedf = pd.DataFrame.spatial.from_geodataframe(gdf)
    else:
        jl_keys = list(json_loaded.keys())
        exc_msg = f"""
        Error! Key value '{k_features}' is not in the list of keys
        in the loaded JSON string, whose keys include {jl_keys}. \nYou may need
        to specify the feature collection by indicating k_features=<feature coll key>.
        """
        raise Exception(exc_msg)

    return sedf

sedf_t = json_to_sedf(txt_test)
sedf_t.head()


,group_index,value,center,SHAPE
0,0,900.0,"[-121.44875188166795, 38.593163819244374]","{""rings"": [[[-121.460692, 38.595053], [-121.45..."


In [20]:
# gpd.GeoDataFrame(polygon_txt['features'])
gdf = gpd.GeoDataFrame.from_features(polygon_features)
sedf = pd.DataFrame.spatial.from_geodataframe(gdf)
sedf.head()

,group_index,value,center,SHAPE
0,0,900.0,"[-121.44875188166795, 38.593163819244374]","{""rings"": [[[-121.460692, 38.595053], [-121.45..."


In [15]:
# src = r'I:\Projects\Darren\PEP\PEP_GIS\PEP_GIS.gdb\test_sr51'
# dest = os.path.join(arcpy.env.scratchGDB, 'TEST_sr51')

# arcpy.management.CopyFeatures(src, dest)

<Result 'C:\\Users\\dconly\\AppData\\Local\\Temp\\scratch.gdb\\TEST_sr51'>

In [2]:
# Make isochrone around multiple points along a line
# https://openrouteservice.org/dev/#/api-docs/isochrones

# input project line feature class
line_fc = r"I:\Projects\Darren\PEP\PEP_GIS\PEP_GIS.gdb\test_sr51"
sref_wgs84 = arcpy.SpatialReference(4326)


# make temporary feature class of points at regular intervales along lines
# FYI, time permitting, the shapely library has some options for doing this that *might* be faster than ESRI tool
temp_pt_fc = os.path.join(arcpy.env.scratchGDB, "TEMP_pts")
arcpy.management.GeneratePointsAlongLines(line_fc, 
                                          temp_pt_fc, "DISTANCE", 
                                          Distance="1000 feet", 
                                          Include_End_Points="END_POINTS")

# calc x/y coords in WGS84 (WKID 4326) for compatibility with ORS API
pt_fl = "pt_fl"
arcpy.MakeFeatureLayer_management(temp_pt_fc, pt_fl)
arcpy.AddGeometryAttributes_management(Input_Features=pt_fl, 
                                       Geometry_Properties=['POINT_X_Y_Z_M'],
                                      Coordinate_System=sref_wgs84)

# print([f.name for f in arcpy.ListFields(temp_pt_fc)])

# make array of points at regular intervals along line to
line_pts = []
with arcpy.da.SearchCursor(pt_fl, ["POINT_X", "POINT_Y"]) as cur:
    for row in cur:
        lon = row[0]
        lat = row[1]
        pt_coords = [lon, lat]
        line_pts.append(pt_coords)
        
# batchify points into groups of 5, because ORS API cannot process more than 5 points in single call

line_pts_batched = [line_pts[i:i+5] for i, v in enumerate(line_pts) if i % 5 == 0]
# line_pts_batched


                    



In [28]:
# generate isochrones around each of those points
max_time_mins = 15
max_time_sec = max_time_mins * 60
travel_mode = "driving-car" # "driving-car" #"foot-walking"

gdf_master = gpd.GeoDataFrame()

# Go through each batch of 5 points and draw an isochrone around them, then combine all the batches together
# into 1 geodatframe with all relevant isochrone polygons in it. Next step would then be dissolve all polygons.
for pts_batch in line_pts_batched:

    body = {"locations":pts_batch, "range":[max_time_sec], "range_type":"time"}

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': ors_api_key,
        'Content-Type': 'application/json; charset=utf-8'
    }

    call = requests.post(f'https://api.openrouteservice.org/v2/isochrones/{travel_mode}', json=body, headers=headers)

    polygon_txt = call.json()  # call.text
    gdf_batch = gpd.GeoDataFrame.from_features(polygon_txt["features"])
    gdf_batch['dissolve_col'] = 0
    gdf_master = gdf_master.append(gdf_batch)
    
# gdf_master.head(14)
# gdf.plot(cmap='Set1')



In [29]:
# Dissolve the multiple polygons into single polygon using Geopandas
# gdf_master.dissolve(by='value')

gdf_master

,geometry,group_index,value,center,dissolve_col
0,"POLYGON ((-121.71354 38.55061, -121.71274 38.5...",0,900.0,"[-121.46455008494888, 38.57984351748153]",0
1,"POLYGON ((-121.71103 38.55118, -121.71024 38.5...",1,900.0,"[-121.46333190871563, 38.58175442000113]",0
2,"POLYGON ((-121.70856 38.55174, -121.70776 38.5...",2,900.0,"[-121.46152551873665, 38.58333858367592]",0
3,"POLYGON ((-121.70612 38.55230, -121.70532 38.5...",3,900.0,"[-121.4593033232335, 38.584466936493016]",0
4,"POLYGON ((-121.70365 38.55286, -121.70286 38.5...",4,900.0,"[-121.45666598300787, 38.58489877496928]",0
0,"POLYGON ((-121.70119 38.55342, -121.70042 38.5...",0,900.0,"[-121.45395359560415, 38.58518610094978]",0
1,"POLYGON ((-121.69887 38.55388, -121.69837 38.5...",1,900.0,"[-121.45124159025234, 38.585473386459114]",0
2,"POLYGON ((-121.69798 38.55400, -121.69748 38.5...",2,900.0,"[-121.44873812280865, 38.58623674816523]",0
3,"POLYGON ((-121.64448 38.56432, -121.64383 38.5...",3,900.0,"[-121.44689470970613, 38.587794235643614]",0
4,"POLYGON ((-121.65028 38.56327, -121.64965 38.5...",4,900.0,"[-121.44607772960302, 38.589843307220825]",0


In [38]:
# Dissolve the multiple polygons into single polygon using ESRI Spatially Enabled DataFrame
# NOTE that this is only because it's proving, as of 10/29/2021, maddeningly difficult to get
# geopandas's .dissolve() method to work. It keeps giving a "LooseVersion" error.

# UPDATE 10/31/2021 - Geopandas dissolve() finally was able to work on Darren's home macbook after doing following:
# Create new, blank conda env > install geopandas 0.10.0 > that's it!


# https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.GeoAccessor.from_layer

# load the gpd geodataframe into esri spatially enabled data frame
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# convert geopandas geodataframe to esri spatially-enabled geodataframe
sedf = GeoAccessor.from_geodataframe(gdf_master, inplace=False, column_name='SHAPE')
sedf.info()

# fl_combdisos = "fl_combdisos"
# sedf.spatial.to_featurelayer(fl_combdisos)
# arcpy.GetCount_management(fl_combdisos)
# arcpy.management.Dissolve(in_features, out_feature_class, {dissolve_field}, {statistics_fields}, {multi_part}, {unsplit_lines})

# temp_fc = os.path.join(arcpy.env.scratchGDB, "TEMP_fc")

# # better way to dissolve the spatially-enabled geodataframe?
# sedf.spatial.to_featureclass(temp_fc)
# arcpy.management.Dissolve(temp_fc, out_diss_poly)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 3
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   group_index   14 non-null     int64   
 1   value         14 non-null     float64 
 2   center        14 non-null     object  
 3   dissolve_col  14 non-null     int64   
 4   SHAPE         14 non-null     geometry
dtypes: float64(1), geometry(1), int64(2), object(1)
memory usage: 672.0+ bytes


In [55]:
# method of dissolving the SEDF to single polygon
# https://developers.arcgis.com/python/api-reference/arcgis.features.manage_data.html#dissolve-boundaries
import arc
fs = sedf.spatial.to_featureset()
arcgis.features.manage_data.dissolve_boundaries()

AttributeError: 'FeatureSet' object has no attribute 'manage_data'

In [18]:
# merge those isochrones together into single isochrone, which represents the "affected area" of a project
# 10/10/2021 - THIS IS A CLUNKY WAY TO DO THIS. For some reason Geopandas dissolve isn't working, so
# current workflow is gdf > geojson > arcpy featureclass > dissolved arcpy feature class.
# ideally could be gdf > dissolved gdf > geojson
import datetime as dt
time_sufx = str(dt.datetime.now().strftime('%Y%m%d_%H%M'))
out_diss_poly = os.path.join(arcpy.env.scratchGDB, f"TEST_combPoly{time_sufx}")

arcpy.env.overwriteOutput = True
temp_gjson = "polys.json"# os.path.join(arcpy.env.scratchFolder, "polys.json")
temp_polys_fc = os.path.join(arcpy.env.scratchGDB, "TEMP_polys")

json_temp = gdf_master.to_file(temp_gjson, driver="GeoJSON")

arcpy.conversion.JSONToFeatures(temp_gjson, temp_polys_fc)
arcpy.management.Dissolve(temp_polys_fc, out_diss_poly)

<Result 'C:\\Users\\dconly\\AppData\\Local\\Temp\\scratch.gdb\\TEST_combPoly20211011_1109'>

In [39]:
# UPDATE 10/31/2021 - Geopandas dissolve() finally was able to work on Darren's home macbook after doing following:
# Create new, blank conda env > install geopandas 0.10.0 > that's it!

import geopandas as gpd

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world = world[['continent', 'geometry']]
world.head()

continents = world.dissolve(by='continent')

# continents.head()

ImportError: the 'read_file' function requires the 'fiona' package, but it is not installed or does not import correctly.
Importing fiona resulted in: DLL load failed: The specified module could not be found.

In [2]:
continents = world.dissolve(by='continent')

In [3]:
continents.head()


,geometry
continent,
Africa,"MULTIPOLYGON (((32.83012 -26.74219, 32.58026 -..."
Antarctica,"MULTIPOLYGON (((-163.71290 -78.59567, -163.712..."
Asia,"MULTIPOLYGON (((120.29501 -10.25865, 118.96781..."
Europe,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3..."
North America,"MULTIPOLYGON (((-61.68000 10.76000, -61.10500 ..."
